In [21]:
import pandas as pd
import openai
import os
import pickle
from openai import AzureOpenAI
import sys
import json

In [19]:
sys.path.append(os.path.join(os.getcwd(), '..'))

In [20]:
from prompts import *

In [2]:
UNIQUE_ID_COLUMN_NAME = "ROW_ID"
UNIQUE_TEXT_COLUMN_NAME = "TEXT"
UNIQUE_LABEL_COLUMN_NAMES = ['sdoh_economics','sdoh_environment']
# Economics (0: None, 1: True[Non-Adverse], 2: False[Adverse])
# Environment (0: None, 1: True[Non-Adverse], 2: False[Adverse])

In [3]:
def retrieve_social_history(df):
    replace_texts = []
    for row_id in df[UNIQUE_ID_COLUMN_NAME]:
        patient = df[df[UNIQUE_ID_COLUMN_NAME] == row_id][UNIQUE_TEXT_COLUMN_NAME].iloc[0]
        social_history_start = patient.lower().find('social history:')
        pos_ends = []
        pos_ends.append(patient.lower().find('family history:'))
        pos_ends.append(patient.lower().find('physical exam'))
        pos_ends.append(patient.lower().find('medications:'))
        pos_ends.append(patient.lower().find('hospital course:'))
        pos_ends.append(patient.lower().find('review of systems:'))
        pos_ends = [x for x in pos_ends if x > social_history_start]
        pos_ends.append(social_history_start+500)
        social_history_end = min(pos_ends)
        replace_texts.append((row_id,patient[social_history_start:social_history_end]))
    texts = pd.DataFrame(replace_texts,columns =[UNIQUE_ID_COLUMN_NAME,UNIQUE_TEXT_COLUMN_NAME])
    
    return texts

In [4]:
#Paths to MIMIC_CSVs
MIMIC_ADMISSION_CSV = "../ahsan_data/ADMISSIONS.csv" #Fill in path/to/file with the path to your MIMIC-III folder
MIMIC_NOTEEVENTS_CSV = "../ahsan_data/NOTEEVENTS.csv" #Fill in path/to/file with the path to your MIMIC-III folder
MIMIC_SBDH = "../ahsan_data/MIMIC-SBDH.csv" #Fill in path/to/file with the path to your MIMIC-SBDH folder

In [5]:
df = pd.read_csv(MIMIC_ADMISSION_CSV)
notes_df = pd.read_csv(MIMIC_NOTEEVENTS_CSV)

/tmp/ipykernel_2959721/2357746518.py:2: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  notes_df = pd.read_csv(MIMIC_NOTEEVENTS_CSV)


In [6]:
#Loading DataFrames for Annotated and Unnanotated MIMIC Notes

newborn_list = df[df["ADMISSION_TYPE"] == "NEWBORN"].SUBJECT_ID.to_list()
discharge_df = notes_df[notes_df['CATEGORY'] == 'Discharge summary']
non_neonatal = discharge_df[~discharge_df['SUBJECT_ID'].isin(newborn_list)]

sbdh_data = pd.read_csv(open(MIMIC_SBDH, 'r+', encoding='UTF-8'),encoding='UTF-8', on_bad_lines='warn')
sbdh_data = sbdh_data.rename(columns={'row_id':UNIQUE_ID_COLUMN_NAME})

annotated_list = sbdh_data[UNIQUE_ID_COLUMN_NAME].tolist()
annotated_notes = discharge_df[discharge_df[UNIQUE_ID_COLUMN_NAME].isin(annotated_list)]
annotated_subjects = discharge_df[discharge_df[UNIQUE_ID_COLUMN_NAME].isin(annotated_list)].SUBJECT_ID.to_list()

no_soc_his = []
for index, row in non_neonatal.iterrows():
    if 'social history:' not in row[UNIQUE_TEXT_COLUMN_NAME].lower():
        no_soc_his.append(row[UNIQUE_ID_COLUMN_NAME])

final_sdoh_list = non_neonatal[~non_neonatal[UNIQUE_ID_COLUMN_NAME].isin(no_soc_his)]
unnanotated_notes = final_sdoh_list[~final_sdoh_list[UNIQUE_ID_COLUMN_NAME].isin(annotated_list)]

annotated_sh = retrieve_social_history(annotated_notes)
annotated_sh = pd.merge(annotated_sh,sbdh_data[[UNIQUE_ID_COLUMN_NAME] + UNIQUE_LABEL_COLUMN_NAMES],on=UNIQUE_ID_COLUMN_NAME, how='left')
unannotated_sh = retrieve_social_history(unnanotated_notes)

df = newborn_list = notes_df = discharge_df = non_neonatal = annotated_list = annotated_subjects = no_soc_his = final_sdoh_list = unnanotated = sbdh_data = None

In [8]:
annotated_sh

,ROW_ID,TEXT,sdoh_economics,sdoh_environment
0,178,Social History:\nPatient recently discharged f...,2,1
1,227,SOCIAL HISTORY: The patient is a home health ...,1,0
2,235,"Social History:\nA retired anesthesiologist, w...",2,0
3,188,Social History:\nLives with his wife. Retired ...,2,1
4,189,SOCIAL HISTORY: Nursing home resident.\n\n,0,1
...,...,...,...,...
7020,58947,SOCIAL HISTORY: Patient raised in [**Location...,2,2
7021,58624,Social History:\nApproximately 15 pack years t...,1,0
7022,58696,Social History:\nHe is Russian speaking. He li...,0,1
7023,59195,Social History:\nPatient lives alone in an [**...,0,1


### Setting up Azure Credentials

In [22]:
with open('../azure_credentials.json', 'r') as file:
    azure_data = json.load(file)
    api_key = azure_data['API_KEY']
    api_version = azure_data['API_VERSION']
    azure_endpoint = azure_data['AZURE_ENDPOINT']
    azure_deployment_name = azure_data['AZURE_DEPLOYMENT_NAME']

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint = azure_endpoint
    )

deployment_name=azure_deployment_name #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment.

# Defining a function to create the prompt from the instruction system message, the few-shot examples, and the current query
def create_prompt(system_message, user_message):    
    formatted_message = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    
    return formatted_message

# This function sends the prompt to the GPT model
def send_message(message, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=message,
        temperature=0,
        max_tokens=max_response_tokens,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    return response.choices[0].message.content.strip()

### Extracting SDoH from data

In [23]:
annotated_sh['sdoh_economics'].value_counts()

sdoh_economics
0    4295
2    1742
1     988
Name: count, dtype: int64

In [44]:
filtered_df = annotated_sh[(annotated_sh['sdoh_economics'] == 1) | (annotated_sh['sdoh_economics'] == 2)]

In [46]:
filtered_df.reset_index(drop=True)

,ROW_ID,TEXT,sdoh_economics,sdoh_environment
0,178,Social History:\nPatient recently discharged f...,2,1
1,227,SOCIAL HISTORY: The patient is a home health ...,1,0
2,235,"Social History:\nA retired anesthesiologist, w...",2,0
3,188,Social History:\nLives with his wife. Retired ...,2,1
4,198,Social History:\nRussian-speaking widow (husba...,2,1
...,...,...,...,...
2725,58064,Social History:\nLives at home with parents. ...,1,1
2726,58259,"Social History:\nsingle, lives with his signfi...",1,1
2727,58947,SOCIAL HISTORY: Patient raised in [**Location...,2,2
2728,58624,Social History:\nApproximately 15 pack years t...,1,0


In [27]:
index_list = []
llm_response_list = []
filtered_df = annotated_sh[(annotated_sh['sdoh_economics'] == 1) | (annotated_sh['sdoh_economics'] == 2)]
filtered_df = filtered_df.reset_index(drop=True)
system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."

start_index = 0

try:
    for index, row in filtered_df[start_index:len(filtered_df)].iterrows():
        free_text = row['TEXT']
        user_message = step1_query_ahsan.format(free_text=free_text)
        openai_message = create_prompt(system_message, user_message)
        response = send_message(openai_message, deployment_name)
        
        index_list.append(index)
        llm_response_list.append(response)
        print(index)
        print(free_text)
        print(response)
        print()
except Exception as err:
    print("Something went wrong: ", err)

llm_employment_adverse_nonadverse_step1 = pd.DataFrame({'index': index_list, 'llm_employment_adverse_nonadverse_step1': llm_response_list})

file_name = 'llm_employment_adverse_nonadverse_step1_' + str(start_index) + '_' + str(len(filtered_df)) + '.pkl'
with open(file_name, 'wb') as file:
    pickle.dump(llm_employment_adverse_nonadverse_step1, file)

0
Social History:
Patient recently discharged from [**Hospital1 **] for severe
depression. Family reports patient was very sad and attempted to
kill himself by wrapping a telephone cord around his neck. Lives
with his elderly wife, worked as a chemist in [**Country 532**].


YES

1
SOCIAL HISTORY:  The patient is a home health aide.  Husband
died three months ago.  She walks but recently was unable to
do so.  She otherwise has a very close family.


YES

2
Social History:
A retired anesthesiologist, worked in pain management. Denies
tobacco, drugs. Bottle of wine per week.


YES

3
Social History:
Lives with his wife. Retired for 30 years, used to own a bagel
shop and was a landlord. Able to ambulate around the home with
assistance and/or walker. Uses a wheelchair outside of the home.
-Tobacco: smoked 1 ppd for 40 yrs, quit about 6 months ago
-EtOH: drinks 3-4 glasses of scotch or wine daily
-Drugs: marijuana ~once weekly


YES

4
Social History:
Russian-speaking widow (husband [**Name